## ESoWC 2019: Data-driven feature selection towards improving forecast-based prediction of wildfire hazard

Team: [@mariajoaosousa](https://github.com/mariajoaosousa), [@eduardogfma](https://github.com/eduardogfma)

Project github page: [https://github.com/esowc/ml_fire](https://github.com/esowc/ml_fire)

# Data partitioning

This notebook focuses on partitioning the datasets into train and test sets. The work was developed using [Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb), a free Jupyter notebook environment that runs entirely in the cloud and enables complete integration with Google Drive.

The note book is structured as follows. Section 1 focuses on a very brief introduction to Colaboratory. In Section 2, data partitionig and cross-validation are discussed. 

# 1. Colaboratory

*The present section focuses on a very brief introduction to [Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) and it is not intended to be taken as an alternative to the official tutorials and documentation.*

Navigationwise, Colaboratory offers an easy way to access specific sections and files available on a virtual disk. On the left the user can find the **Table of contents** and the **Files** tabs. The **Code snnipets** tab is a convinient way of *copying and pasting* already developed code for standard operations.

For this particular project, however, there are two particular features of the Colaboratory environment that should be stressed. Namely, the possibility of:

* making use of GPU/TPU for parallell copmuting,
* installing additional python dependencies,
* mouting the Google Drive on the virtual disk, enabling the user to easily import/export data.

## 1.1. Runtime type

To define or redefine the Runtime type in Colaboratory, the user must simply go to *Runtime -> Change runtime type* and select the desired properties.

Regarding the hardware accelarator options, it should be noted they influence the available session's memory.

## 1.2. Installing additional dependencies

The Colaboratory environment is built on Linux. Thus it is possible to access the console by using the `!` prefix. The next code cell intstalls the `netcdf4` library in the current python environment.

In [0]:
!pip install netcdf4

The most used libraries are already installed by default. However, in some non-standard situations it may be necessary to install several dependencies. In that case, there exists also the possibility of installing anaconda. Since this notebook does not require such preparations, we skip that topic for now with the promisse to revisit it in the future, if necessary.

## 1.3. Mounting Google Drive

To be able to access files from Google Drive, it must be first mounted. For security reasons this operation requires the user to introduce an authorization code. The process is quick and easy: follow the presented link, copy the code and pasted into the input space (the rectangle presented on the code cell output).

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 2. Processing data

Supervised learning is characterized by providing the *features*, $X$, and *labels*, $y$ (i.e. the target values) to the model. In this framework, the modelling process is formulated as an optimization problem where the error between the predicted output, $\hat{y}$, and the true output, $y$, must be minimised.

The modelling process is usually called *training*. In turn, to evaluate the model's performance, a set of tests must be performed. This process is called *testing*. In this phase, two things must be assessed:

1. how good is the model's prediction, i.e. how close are $\hat{y}$ and $y$?
2. how generic is the model, i.e. provided new data is the model able to deliever good predictions?

Learning a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data. This situation is called **overfitting**. To avoid it, in supervised learning one needs a *training set* and a *test set* [[1](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation)]. Where each set contains both features and labels. A third set may also be provided, the *validation set* -- to be used during training and provide information about the model's learning.

This section focuses on data partitioning, i.e. creating the train and test datasets from the available raw data. The data used comprises 3 datasets -- FWI, BUI, ISI. (For more information about the used data, refer to the [Getting Started With Data tutorials](https://github.com/esowc/ml_fire/tree/master/GettingStartedWithData).) Each one of these datasets are loaded as `xarray.Dataset` and then stored in a dictionary (`rawData`) for easy access.

In [0]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sklearn.model_selection import KFold
%matplotlib inline

In [0]:
rawData ={}

In [0]:
rawData["fwi"] = xr.open_dataset("../content/gdrive/My Drive/ESoWC/datasets/segmented_data/GADM/fwiCAN_GADM00.nc")
rawData["fwi"]

<xarray.Dataset>
Dimensions:  (lat: 45, lon: 162, time: 14061)
Coordinates:
  * lat      (lat) float32 73.33297 72.63123 71.92949 ... 43.15779 42.45604
  * lon      (lon) float32 -149.76562 -149.0625 ... -37.265625 -36.5625
  * time     (time) datetime64[ns] 1980-01-01 1980-01-02 ... 2018-06-30
Data variables:
    fwi      (time, lat, lon) float32 ...
Attributes:
    CDI:               Climate Data Interface version 1.8.2 (http://mpimet.mp...
    history:           Fri Aug 31 16:18:06 2018: cdo cat /hugetmp/fire/geff/r...
    Conventions:       CF-1.6
    Reference date:    19800101
    ECMWF fire model:  2.2
    Lincense:          Copernicus
    version:           2.2
    NCO:               4.6.7
    CDO:               Climate Data Operators version 1.8.2 (http://mpimet.mp...

In [0]:
rawData["isi"] = xr.open_dataset("../content/gdrive/My Drive/ESoWC/datasets/segmented_data/GADM/isiCAN_GADM00.nc")
rawData["bui"] = xr.open_dataset("../content/gdrive/My Drive/ESoWC/datasets/segmented_data/GADM/buiCAN_GADM00.nc")

In [0]:
rawData.keys()

dict_keys(['fwi', 'isi', 'bui'])

## 2.1. Cross validation

*The contents of this section is based on the scikit-learn official [documentation](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation).*

By partitioning the available (raw) data into 3 sets -- train, test, validation -- the number of sample which can be used for learning are drastically reduced, and the results can depend on  a particular random choice for the pair of (train, validation) sets.

To mittigate this, one can use [cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) (CV). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV. In the basic approach, called **k-fold cross-validation**, the training set is split into $k$ smaller sets. The following procedure is followed for each of the $k$ “folds”:

* a model is trained using $k-1$ of the folds as training data,
* the resulting model is validated on the remaining part of the data (i.e. it is used as a test set to compute a performance measure such as accuracy).

One round of cross-validation involves partitioning a sample of data into complementary subsets, performing the analysis on one subset (called the training set), and validating the analysis on the other subset (called the validation set or testing set) [[2](https://en.wikipedia.org/wiki/Cross-validation_(statistics))]. The process can be graphically represented as follows.

![cross-validation_fig1](https://upload.wikimedia.org/wikipedia/commons/1/1c/K-fold_cross_validation_EN.jpg)

Figure 1 -- Cross-validation process.

The performance measure reported by k-fold cross-validation is then the average of the values computed in the loop, as shown in Figure 2. This approach can be computationally expensive, but does not waste too much data (as is the case when fixing an arbitrary validation set).

![](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

Figure 2 -- k-fold cross-validation performance measure.

## 2.1.2. Data partitioning

In this section, the functions to partition data in accordance to the cross-validation method are defined. The pseudocode is the following:



```python
for each dataset in rawData:
  partitionedData = {}  # define dictionary
  i = 1
  for each fold:
    data = {}
    for each feature
      partition dataset into train_data and test_data
      data[feature] = {"train": train_data, "test":test_data}
    partitionedData["fold_{}".format(i)] = data
    i++
  return  partitionedData
```



___
###  *Handle functions*

In [0]:
def part(name, train_ind, test_ind):
  '''
    Cross-validation data partitioning
    
      name: ..................... [string] feature name
      train_ind: ................ [list] indeces to partition data
      test_ind: ................. [list] indeces to partition data
      
      > output: [xr.DataSet] train and test datasets
      
      Given the indeces, this function partitions data into train and test
      datasets. The outputs are xr.DataSet segmented obejcts.
  '''
  
  train, test = rawData[name].isel({'time':train_ind}), rawData[name].isel({'time':test_ind})
  
  return train, test
  
  
def printInfo(name, train, test):
  '''
    Prints information abou the partitioned data
    
      name: ..................... [string] feature name
      train: .................... [xr.DataArray] train data
      test: ..................... [xr.DataArray] test data
      
      > output: void
      
      Prints how many xr.DataArray are comprised in each partitioned dataset
  '''
  
  print("\n  {}_train -- Number of xarrays: ".format(name), len(train.time))
  print("  {}_test -- Number of xarrays: ".format(name), len(test.time))

In [0]:
def cvPartition(n_folds, inputs, output):
  '''
    Partitions data for cross-validation
    
    inputs: ..................... [string] list of features
    output: ..................... [strings] list of labels
    
    > output: [dict] partitionedData
    
    Provided the features and labels, the function outputs a dictionary
    containing the partitioned data by fold. The structure of the dictionary 
    is the  following.
    
    Fold 1
      |
      -- bui    ----> input
      .  |
      .  -- train
      .  |
      .  -- test
      |
      -- isi    ----> input
      .  |
      .  -- train
      .  |
      .  -- test
      .
      .
      
    (...)
      
      .
      .
      |
      -- fwi    ----> output
      .  |
      .  -- train
      .  |
      .  -- test
      
    (...)
      
      Fold 2
      |
      -- bui
      .  |
      .  -- train
      .  |
      .  -- test
      |
      -- isi
      .  |
      .  -- train
      .  |
      .  -- test
      
    (...)
    
  '''
  
  # initialisation
  partitionedData = {}            
  i = 1
  x = rawData[inputs[0]][inputs[0]]
  inputs.append(output)
  
  # start partition process
  for train_index, test_index in KFold(n_splits=n_folds).split(X=x):

    print(" ---------------- Fold ", i,"---------------- ")
    print("TRAIN:", train_index, "\nTEST: ", test_index)


    # store data  
    data = {}

    for feat in inputs:
      train, test = part(feat, train_index, test_index)  # partion data
      data[feat] = {"train": train, "test": test}
      
      # show info
      printInfo(feat, train, test)  

    print("\n")

    partitionedData["fold_{}".format(i)] = data

    i +=1
    
  return partitionedData

___
### *Partitioning data*

With the data already partitioned, two examples are shown below. The first one partitions data into 5 folds, while the second one defines 10 folds.

#### Example 1: using 5 folds

In [0]:
n_folds = 5
inputs = ["bui","isi"]
output = "fwi"

partitionedData_5Fold = cvPartition(n_folds, inputs, output)

 ---------------- Fold  1 ---------------- 
TRAIN: [ 2813  2814  2815 ... 14058 14059 14060] 
TEST:  [   0    1    2 ... 2810 2811 2812]

  bui_train -- Number of xarrays:  11248
  bui_test -- Number of xarrays:  2813

  isi_train -- Number of xarrays:  11248
  isi_test -- Number of xarrays:  2813

  fwi_train -- Number of xarrays:  11248
  fwi_test -- Number of xarrays:  2813


 ---------------- Fold  2 ---------------- 
TRAIN: [    0     1     2 ... 14058 14059 14060] 
TEST:  [2813 2814 2815 ... 5622 5623 5624]

  bui_train -- Number of xarrays:  11249
  bui_test -- Number of xarrays:  2812

  isi_train -- Number of xarrays:  11249
  isi_test -- Number of xarrays:  2812

  fwi_train -- Number of xarrays:  11249
  fwi_test -- Number of xarrays:  2812


 ---------------- Fold  3 ---------------- 
TRAIN: [    0     1     2 ... 14058 14059 14060] 
TEST:  [5625 5626 5627 ... 8434 8435 8436]

  bui_train -- Number of xarrays:  11249
  bui_test -- Number of xarrays:  2812

  isi_train -- Nu

#### Example 2: using 10 folds

In [0]:
n_folds = 10
inputs = ["bui","isi"]
output = "fwi"

partitionedData_10Fold = cvPartition(n_folds, inputs, output)

 ---------------- Fold  1 ---------------- 
TRAIN: [ 1407  1408  1409 ... 14058 14059 14060] 
TEST:  [   0    1    2 ... 1404 1405 1406]

  bui_train -- Number of xarrays:  12654
  bui_test -- Number of xarrays:  1407

  isi_train -- Number of xarrays:  12654
  isi_test -- Number of xarrays:  1407

  fwi_train -- Number of xarrays:  12654
  fwi_test -- Number of xarrays:  1407


 ---------------- Fold  2 ---------------- 
TRAIN: [    0     1     2 ... 14058 14059 14060] 
TEST:  [1407 1408 1409 ... 2810 2811 2812]

  bui_train -- Number of xarrays:  12655
  bui_test -- Number of xarrays:  1406

  isi_train -- Number of xarrays:  12655
  isi_test -- Number of xarrays:  1406

  fwi_train -- Number of xarrays:  12655
  fwi_test -- Number of xarrays:  1406


 ---------------- Fold  3 ---------------- 
TRAIN: [    0     1     2 ... 14058 14059 14060] 
TEST:  [2813 2814 2815 ... 4216 4217 4218]

  bui_train -- Number of xarrays:  12655
  bui_test -- Number of xarrays:  1406

  isi_train -- Nu

As can be seen, data was correctly partitioned. 

In summary, the data partioning consisted ont the following:

* a dictionary to store raw data was instanciated:
>`rawData = {"bui": bui_data, "isi": isi_data, "fwi": fwi_data}`,

 
* from `rawData`, data was then partitioned and stored in a new dictionary:
>`partitionedData = {"bui": bui_Test&Train_data, "isi": isi_Test&Train_data, "fwi": fwi_Test&Train_data}`.